In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import os
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import datetime
from datetime import date, timedelta
e_date = datetime.datetime.strptime('2020-06-20', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2020-04-01', '%Y-%m-%d')

data_path = os.path.join('..','OD')

In [2]:
delta = e_date - i_date

target_days = []
for i in range(delta.days + 1):
    day = i_date + timedelta(days=i)
    target_days.append(day)

In [3]:
def read_OD_fn(source_type):
    ods = {}
    for date in tqdm_notebook(target_days):
        od_date= pd.read_csv(os.path.join(data_path, 'date_{}_OD_{}.csv'.format(source_type,date.strftime('%Y-%m-%d'))), 
                            index_col=0)
        ods[date] = od_date
    return ods

In [4]:
ine_ods = read_OD_fn('INE')

  0%|          | 0/81 [00:00<?, ?it/s]

In [5]:
twt_ods = read_OD_fn('TWT')

  0%|          | 0/81 [00:00<?, ?it/s]

# Serializar los datos

In [6]:
lista_twt1 = []
lista_twt2 = []
lista_twt3 = []
lista_twt4 = []
lista_twt5 = []

for date in tqdm_notebook(target_days):
    lista_twt1.append(twt_ods[date].loc['18GU','total'])
    lista_twt2.append(twt_ods[date].loc['013B','total'])
    lista_twt3.append(twt_ods[date].loc['021S','total'])
    lista_twt4.append(twt_ods[date].loc['219M','total'])
    lista_twt5.append(twt_ods[date].loc['01CA','total'])
    
    
lista_ine1 = []
lista_ine2 = []
lista_ine3 = []
lista_ine4 = []
lista_ine5 = []

for date in tqdm_notebook(target_days):
    lista_ine1.append(ine_ods[date].loc['18GU','total'])
    lista_ine2.append(ine_ods[date].loc['013B','total'])
    lista_ine3.append(ine_ods[date].loc['021S','total'])
    lista_ine4.append(ine_ods[date].loc['219M','total'])
    lista_ine5.append(ine_ods[date].loc['01CA','total'])
    

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

In [7]:
df_twt1 = pd.DataFrame (lista_twt1, columns = ['Viajes'], index= target_days)
df_ine1 = pd.DataFrame (lista_ine1, columns = ['Viajes'], index= target_days)

df_twt2 = pd.DataFrame (lista_twt2, columns = ['Viajes'], index= target_days)
df_ine2 = pd.DataFrame (lista_ine2, columns = ['Viajes'], index= target_days)

df_twt3 = pd.DataFrame (lista_twt3, columns = ['Viajes'], index= target_days)
df_ine3 = pd.DataFrame (lista_ine3, columns = ['Viajes'], index= target_days)

df_twt4 = pd.DataFrame (lista_twt4, columns = ['Viajes'], index= target_days)
df_ine4 = pd.DataFrame (lista_ine4, columns = ['Viajes'], index= target_days)

df_twt5 = pd.DataFrame (lista_twt5, columns = ['Viajes'], index= target_days)
df_ine5 = pd.DataFrame (lista_ine5, columns = ['Viajes'], index= target_days)

In [8]:
lista_twt = []
lista_ine = []

df_twt = df_twt1 + df_twt2 + df_twt3 + df_twt4 + df_twt5
df_ine = df_ine1 + df_ine2 + df_ine3 + df_ine4 + df_ine5

In [9]:
df_twt.head()

,Viajes
2020-04-01,0
2020-04-02,0
2020-04-03,0
2020-04-04,0
2020-04-05,0


In [10]:
df_ine.head()

,Viajes
2020-04-01,3314
2020-04-02,3220
2020-04-03,3934
2020-04-04,3141
2020-04-05,2520


In [11]:
del twt_ods
del ine_ods

# Visualización de datos

In [12]:
print(df_twt.index.min())
print(df_ine.index.max())

2020-04-01 00:00:00
2020-06-20 00:00:00


In [13]:
print(len(df_twt['2020']))

81


In [14]:
print(len(df_ine['2020']))

81


In [15]:
df_twt.describe()

,Viajes
count,81.000000
mean,1.148148
std,1.467235
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,6.000000


In [16]:
df_ine.describe()

,Viajes
count,81.000000
mean,4188.802469
std,922.684743
min,2282.000000
25%,3503.000000
50%,4243.000000
75%,4933.000000
max,5827.000000


### Cointegration test

In [17]:
trips_df = df_twt.copy()
trips_df['ine_trips']= df_ine['Viajes']
trips_df.head()

,Viajes,ine_trips
2020-04-01,0,3314
2020-04-02,0,3220
2020-04-03,0,3934
2020-04-04,0,3141
2020-04-05,0,2520


In [18]:
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    
    def adjust(val, length= 6): 
        return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)


cointegration_test(trips_df)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Viajes ::  13.94     > 12.3212   =>   True
ine_trips ::  1.66      > 4.1296    =>   False


### Granger test

In [19]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(trips_df, variables = trips_df.columns)

,Viajes_x,ine_trips_x
Viajes_y,1.0000,0.1753
ine_trips_y,0.0981,1.0000


### Check stationarity

In [20]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

adfuller_test(trips_df['Viajes'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.446
 No. Lags Chosen       = 0
 Critical value 1%     = -3.515
 Critical value 5%     = -2.898
 Critical value 10%    = -2.586
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [21]:
adfuller_test(trips_df['ine_trips'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -0.4994
 No. Lags Chosen       = 12
 Critical value 1%     = -3.53
 Critical value 5%     = -2.905
 Critical value 10%    = -2.59
 => P-Value = 0.8921. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


In [22]:
trips_df_differenced = trips_df.diff().dropna()

In [23]:
adfuller_test(trips_df_differenced['Viajes'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -8.5803
 No. Lags Chosen       = 2
 Critical value 1%     = -3.518
 Critical value 5%     = -2.9
 Critical value 10%    = -2.587
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [24]:
adfuller_test(trips_df_differenced['ine_trips'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.9358
 No. Lags Chosen       = 12
 Critical value 1%     = -3.532
 Critical value 5%     = -2.906
 Critical value 10%    = -2.59
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


### Select order for VAR Model

In [25]:
trips_df_differenced.columns

Index(['Viajes', 'ine_trips'], dtype='object')

In [26]:
model = VAR(trips_df_differenced)

In [27]:
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,14.28,14.34,1.589e+06,14.30
1,14.18,14.38,1.441e+06,14.26
2,13.89,14.22*,1.077e+06,14.02*
3,13.92,14.37,1.108e+06,14.10
4,13.97,14.56,1.170e+06,14.20
5,13.84,14.56,1.034e+06,14.13
6,13.70,14.55,8.997e+05,14.04
7,13.68,14.66,8.833e+05,14.06
8,13.66,14.77,8.729e+05,14.10
9,13.65,14.90,8.779e+05,14.15


Best lag is set to 7 days

In [28]:
model_fitted = model.fit(7)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 21, Sep, 2021
Time:                     11:45:16
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    14.6057
Nobs:                     73.0000    HQIC:                   14.0395
Log likelihood:          -675.916    FPE:                    870023.
AIC:                      13.6644    Det(Omega_mle):         598702.
--------------------------------------------------------------------
Results for equation Viajes
                  coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------
const                0.027269         0.197953            0.138           0.890
L1.Viajes           -0.645437         0.128963           -5.005           0.000
L1.ine_trips        -0.000360         0.000423           -0.851

### Forecast test

In [29]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


def compute_metrics_fn(y_valid, y_hat):
    mae_ = mean_absolute_error(y_valid, y_hat)
    mse_ = mean_squared_error(y_valid, y_hat)
    rmse_ = mean_squared_error(y_valid, y_hat, squared = False)
    cvrmse_ = rmse_/np.mean(y_valid)*100 # it is a percentage
    mape_ = mape(y_valid, y_hat)
    
    return mae_, mse_, rmse_, cvrmse_, mape_

trips_df.shape

(81, 2)

In [30]:
ine_true = []
ine_hat = []
for i in range(60, 81-7):
    X= trips_df.iloc[0:i]
    X_differenced = X.diff().dropna()
    
    y =trips_df.iloc[i:i+7]
    y_true_ine = [v[1] for v in y.values]
    
    model = VAR(X_differenced.iloc[:-7])
    model_fitted = model.fit(7)
    y_hat = model_fitted.forecast(y=X_differenced.values[-7:], steps=7)
    
    df_forecast = pd.DataFrame(y_hat, index=trips_df.iloc[i:i+7].index, columns=trips_df.columns + '_1d')

    columns = trips_df.columns
    for col in columns:  
        df_forecast[str(col)+'_forecast'] = X[col].iloc[-1] + df_forecast[str(col)+'_1d'].cumsum()

    y_hat_ine = list(df_forecast['ine_trips_forecast'].values)
    
    ine_true = ine_true + y_true_ine
    ine_hat = ine_hat + y_hat_ine

testScore_MAE, testScore_MSE, testScore_RMSE, testScore_CVRMSE, testScore_MAPE  =  compute_metrics_fn(ine_true, ine_hat)

print('Resultado del test : %.2f MAE' % ( testScore_MAE))
print('Resultado del test: %.2f MSE' % ( testScore_MSE))
print('Resultado del test: %.2f RMSE' %  (testScore_RMSE))
print('Resultado del test: %.2f CVRMSE' % ( testScore_CVRMSE))
print('Resultado del test: %.2f MAPE \n' % (testScore_MAPE))

Resultado del test : 399.68 MAE
Resultado del test: 261084.07 MSE
Resultado del test: 510.96 RMSE
Resultado del test: 9.92 CVRMSE
Resultado del test: 8.34 MAPE 



In [31]:
for i in range(0,7):
    y_true = ine_true[i::7]
    y_hat = ine_hat[i::7]
    testScore_MAE, testScore_MSE, testScore_RMSE, testScore_CVRMSE, testScore_MAPE  =  compute_metrics_fn(y_true, y_hat)

    print('Resultado del test dia %d: %.2f MAE' % (i, testScore_MAE))
    print('Resultado del test dia %d: %.2f MSE' % (i, testScore_MSE))
    print('Resultado del test dia %d: %.2f RMSE' % (i, testScore_RMSE))
    print('Resultado del test dia %d: %.2f CVRMSE' % (i, testScore_CVRMSE))
    print('Resultado del test dia %d: %.2f MAPE \n' % (i, testScore_MAPE))

print("That's all folks!")

Resultado del test dia 0: 523.13 MAE
Resultado del test dia 0: 346656.11 MSE
Resultado del test dia 0: 588.78 RMSE
Resultado del test dia 0: 11.43 CVRMSE
Resultado del test dia 0: 10.53 MAPE 

Resultado del test dia 1: 455.69 MAE
Resultado del test dia 1: 328843.36 MSE
Resultado del test dia 1: 573.45 RMSE
Resultado del test dia 1: 11.18 CVRMSE
Resultado del test dia 1: 9.51 MAPE 

Resultado del test dia 2: 355.45 MAE
Resultado del test dia 2: 243005.73 MSE
Resultado del test dia 2: 492.96 RMSE
Resultado del test dia 2: 9.58 CVRMSE
Resultado del test dia 2: 7.58 MAPE 

Resultado del test dia 3: 336.42 MAE
Resultado del test dia 3: 216370.43 MSE
Resultado del test dia 3: 465.16 RMSE
Resultado del test dia 3: 9.07 CVRMSE
Resultado del test dia 3: 7.18 MAPE 

Resultado del test dia 4: 376.03 MAE
Resultado del test dia 4: 223186.82 MSE
Resultado del test dia 4: 472.43 RMSE
Resultado del test dia 4: 9.18 CVRMSE
Resultado del test dia 4: 7.83 MAPE 

Resultado del test dia 5: 382.64 MAE
Resul